
# Workshop about controllers

## Contents
* Controller
* Writing controller - general
* Writing motor controller - XYZ Stage
* Writing pseudo motor controller - Mirror Table
* Writing counter/timer controller - Network Trafic Counter
* Deployment and packaging
* Miscellaneous topics

## Controller
* [Controller overview](http://www.sardana-controls.org/en/latest/devel/overview/overview_controller.html#sardana-controller-overview)
* [Writing controllers](http://www.sardana-controls.org/en/latest/devel/howto_controllers/index.html)
* [Controller API reference](http://www.sardana-controls.org/en/latest/devel/api/api_controller.html#sardana-controller-api)

### Controller overview

* Maps the communication between a pool element e.g. motor and the underlying hardware (example: a motor controller crate)
* Element axis refers to the ID of a specific hardware object (like a motor) with respect to its controller
* 
The controller object is also exposed as a Tango device.

### Writing controllers
* Before writing a new controller check in the [third-party repository](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/) if someone already wrote a similar one
* If not, it does not harm to ask on one of the communication channels e.g. mailing list, github issue, etc.

### Writing controllers - general
* Constructor
 * Called on: controllers creation, pool startup and controller's code reload
 * Accepts arguments: instance (name of the controller instance) and properties (dictionary with the controller properties)
 * If an exception is raised when constructing the controller, the controller automatically gets into the Fault state and its status describes the exception that occured

### Writing controllers - general
* AddDevice and DeleteDevice
 * Called on: element creation/deletion, pool start/stop and controller's code reload

### Writing controllers - general
* Get axis state (State sequence)
 * Applies only to the to physical elements
 * Called on: state request, during operations e.g. motion, acquisition
 * Returns: state and optionally status, if no status is returned, it will be composed by Sardana from the state (in case of motor also returns limit switches)
 * If an exception is raised when reading the state, the axis automatically gets into the Fault state and the status contains the exception details.

### Writing controllers - general
* Axis extra attributes - attributes that are not included in the standard inteface e.g. close loop for a motor
* Controller extra attributes - attributes that are not included in the standard interface e.g. 
* Controller properties - similar to attributes but are foreseen for more static characteristics e.g. communication host and port

### Writing controllers - general
* Controller has its own logger - `_log` member and its level can be controller with an attribute of the controller
* When developing controllers, it is very useful to call `ReloadControllerClass` and `ReloadControllerLib` commands of the pool - see [#53](https://github.com/sardana-org/sardana/issues/53)

### Synchronized start
```
/FOR/ Each controller(s) implied in the motion
     - Call PreStartAll()
/END FOR/
/FOR/ Each motor(s) implied in the motion
     - ret = PreStartOne(motor to move, new position)
     - /IF/ ret is not true
        /RAISE/ Cannot start. Motor PreStartOne returns False
     - /END IF/
     - Call StartOne(motor to move, new position)
/END FOR/
/FOR/ Each controller(s) implied in the motion
     - Call StartAll()
/END FOR/
```

### Synchronized start

[Single motor start sequence diagram](http://www.sardana-controls.org/en/latest/devel/api/api_motor.html#motion)

### Optimized hardware access while reading multiple axes (also state)

```
/FOR/ Each controller(s) implied in the reading
     - Call PreReadAll()
/END FOR/
/FOR/ Each motor(s) implied in the reading
     - PreReadOne(motor to read)
/END FOR/
/FOR/ Each controller(s) implied in the reading
     - Call ReadAll()
/END FOR/
/FOR/ Each motor(s) implied in the reading
     - Call ReadOne(motor to read)
/END FOR/
```

### Optimized hardware access while reading multiple axes (also state)

[Single motor read sequence diagram](http://www.sardana-controls.org/en/latest/devel/api/api_motor.html#motor-position)

### [How to write motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html)

* Get motor state - `StateOne`:
 * return status of the limit switches (home, upper, lower)
 * should become Alarm if any of the overtravel limit switches becomes active - see discussion in [#507](https://github.com/sardana-org/sardana/issues/507)
 * is called multiple times during the motion operation

### [How to write motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html)

* Get motor position - `ReadOne`
 * return dial position (dial position = motor position / steps per unit)
 * is called multiple times during the motion operation


### [How to write motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html)
* Move a motor - `StartOne`:
 * Accepts argument: dial position (motor position = dial position * steps per unit)

### [How to write motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html)

* Stop a motor - `StopOne`:
 * Gracefully stop a motor (deceleration and base rate should be respected)
 * Stopping multiple axis with one access to the hardware, if possible, should be allowed - see [#157](https://github.com/sardana-org/sardana/issues/157)
* Abort a motor - `AbortOne`:
 * Gracefully stop a motor (decelration and base rate should be respected)
 * Aborting multiple axis with one access to the hardware, if possible, should be allowed - see [#157](https://github.com/sardana-org/sardana/issues/157)

### [How to write motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html)
* Standard axis attributes - `SetAxisPar` and `GetAxisPar`
  * acceleration, deceleration, velocity, base rate and steps per unit
  * some combinations of parameters may not be coherent what to do then - see [#30](https://github.com/sardana-org/sardana/issues/30) and [#420](https://github.com/sardana-org/sardana/pull/420)

### [How to write motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_motorcontroller.html)

* Define motor position - `DefinePosition`:
 * Loads the new motor position to the hardware
* Timestamp read position - return `SardanaValue` object containing the position and the timestamp in the `ReadOne`
* Examples: IcePAP, Pmac, Tango attribute

### Example - XYZ Stage

### [How to write pseudo motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_pseudomotorcontroller.html)

* Define class members `pseudo_motor_roles` and `motor_roles`
 * Some use cases (advanced), for example HKL, may require dynamic roles - see [#86](https://github.com/sardana-org/sardana/issues/86)
 * If controller comprises only one pseduo motor the `pseudo_motor_roles` can be omitted
* Calculate pseudo motor position - `CalcPseudo`
 * Accepts: axis, physical motor positions and the current pseudo motor positions as arguments
 * Returns calculated pseudo motor position
 * The current pseudo motor positions that arrives to `CalcPseudo` are set values and not read values TODO

### [How to write pseudo motor controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_pseudomotorcontroller.html)
* Calculate physical motor position - `CalcPhysical`
 * Accepts: pseudo motor positions and the current physical motor positions as arguments
 * Returns calculated physical motor position
* If great performance gain can be achived it is possible to use `CalcAllPseudo` and `CalcAllPhysical` methods
 * The default implementation iterates over axes and call the `CalcPseudo` and `CalcPhysical` methods multiple times
* Examples: Mirrors, ID, Tables, Energy, Twin motors

### Example: Mirror Table

`defctrl MirrorVerticalPseudoMotorController mvert mzc=mzc mzl=mzl mzr=mzr z=mz pitch=mpitch roll=mroll dim_x 261 dim_y 1262.5`

### [How to write counter/timer controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_countertimercontroller.html)
* Load a counter - `LoadOne`
 * receives either integration time or monitor counts (negative number) as arguments \*
 * is called only on the controller's timer or monitor
* Read a counter value - `ReadOne` \*
 * returns a single counter value
 * is called multiple times during the motion acquisition
* Start a counter - `StartOne`
 * receives intergation timer or monitor count as argument
* Stop and abort a counter - similar to stopping/aborting a motor
 
\* An advanced API is available for continuous acquisitions e.g. continuous scans

### [How to write counter/timer controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_countertimercontroller.html)
* Select timer/monitor - `timer` and `monitor` controller parameters set with `SetCtrlPar` (timer/monitor axis number)
* Select acquisition mode - `acquisition_mode` controller parameter set with `SetCtrlPar` (`AcqMode` enumeration)
* Timestamp read counter value - return `SardanaValue` object containing the counter value and the timestamp in the `ReadOne`

### [How to write counter/timer controller](http://www.sardana-controls.org/en/latest/devel/howto_controllers/howto_countertimercontroller.html) - continuous acquisition

* Select synchronization mode - `synchronization` controller parameters set with `SetCtrlPar` (`AcqSynch` enumeration)
* Load a counter - `LoadOne`
 * receives number of repetitions as argument (if software synchronizer is used the repetitions is set to 1)
* Read a counter value - `ReadOne`
 * progressively returns counter values in bunches while the acquisition is in progress
 * skipping the *online readout* may be achieved on the controller by simply conditioning the hardware readout with the state different than Moving
 
All the controller parameters are configured on the measurement group configuration so they may vary between measurement groups. Applying of these parameters is done at the beginning of the acquisition.
 
* Examples: AlbaEM, Adlink2005, NI6602

### Example: Network Trafic Counter

### Controllers deployment at ALBA

The controllers are deployed at three levels:

* User - created by the scientists (developed directly in the NFS mounted directory, no revision control)
* Control engineer - created by the control engineers (repository check outs)
* Production - installed via packages

### Controllers deployment at ALBA


#### PoolPath property now (OpenSUSE + bliss)
``` 
/beamlines/bl22/controls/user_ctrls  # User
/beamlines/bl22/controls/devel/poolcontrollers # Control engineer
/homelocal/sicilia/lib/python/site-packages/poolcontrollers # Package
/homelocal/sicilia/lib/python/site-packages/poolcontrollers/IcePAPCtrl
/homelocal/sicilia/lib/python/site-packages/poolcontrollers/IBACtrl
```

### Controllers deployment at ALBA


#### PoolPath property future (debian)
```
/beamlines/bl22/controls/user_ctrls # User
/beamlines/bl22/controls/devel/poolcontrollers # Control engineer
/usr/lib/sardana/controllers # Package
/usr/lib/sardana/controllers/IcePAPCtrl
/usr/lib/sardana/controllers/IBACtrl
```

## Advanced topics

### Use of taurus vs. use of PyTango?

Taurus pros:

* More friendly API
* A lot of work is already done e.g. configuration events
* Take care with the pythonic write

### Use of taurus vs. use of PyTango?

PyTango pros:
* Lightweigh
* Access to full Tango API e.g. Group, asychronous read, etc.
* Allow pythonic write

In [4]:
import time, taurus, PyTango
mot_taurus = taurus.Device("mot01")
mot_tango = PyTango.DeviceProxy("mot01")
print "taurus:", mot_taurus.position, "tango:", mot_tango.position
mot_taurus.position = 10  # this does not move the motor!!!
print "taurus:", mot_taurus.position, "tango:", mot_tango.position
mot_tango.position = 0
time.sleep(3)
print "taurus:", mot_taurus.position, "tango:", mot_tango.position

taurus: 10 tango: 10.0
taurus: 10 tango: 10.0
taurus: 10 tango: 0.0


### Using external modules

* Controllers may use external modules
* They must accessible either via:
 * `PythonPath` property of the Pool device
 * `PYTHONPATH` environment variable
* The modules accessible via `PythonPath` property have precedence over the ones available in Python.

### Class inheritance

* The controller inheritance is possible
 * From another controller class
 * From an arbitrary class
* Base classes must be importable - see "Using external modules" slide
* If all base class and the controller class are located at the same path the controller manager can fail due to the uncertain loading order

### Class inheritance - examples:
* [TurboPmacController](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/motor/PmacCtrl/TurboPmacCtrl.py)(Controller)
 * [LtpTurboPmacController](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/motor/PmacCtrl/AlbaLtpTurboPmacCtrl.py)(Controller)

* [Ni660XCtrl](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/countertimer/Ni660XCtrl/Ni660XCTCtrl.py)(Base class) 
 * [Ni660XPositionCTCtrl](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/countertimer/Ni660XCtrl/Ni660XPositionCTCtrl.py)(Controller)
 * [Ni660XCounterCTCtrl](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/countertimer/Ni660XCtrl/Ni660XCounterCTCtrl.py)(Controller)
 * [Ni660XPulseWidthCTCtrl.py ](https://sourceforge.net/p/sardana/controllers.git/ci/master/tree/python/countertimer/Ni660XCtrl/Ni660XPulseWidthCTCtrl.py)(Controller)